In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import cv2
import os
from tensorflow.keras.utils import load_img,img_to_array

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
path = '/content/drive/MyDrive/kaggle/ECG_Images/train'

In [4]:
np.random.seed(1234)

# Get the list of categories in the training directory
img_list = os.listdir(path)

# Define labels for the categories
label_dict = {
    'ECG Images of Myocardial Infarction Patients (240x12=2880)': 0,
    'ECG Images of Patient that have History of MI (172x12=2064)': 1,
    'ECG Images of Patient that have abnormal heartbeat (233x12=2796)': 2,
    'Normal Person ECG Images (284x12=3408)': 3
}

data = []
label = []

# Loop through each category folder and read images
for cat in img_list:
    C = 0
    pic_list = os.path.join(path, cat)
    for img in os.listdir(pic_list):
        image_path = os.path.join(pic_list, img)
        image = cv2.imread(image_path)

        # Convert image to RGB and resize it
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        # Append image and label to respective lists
        data.append(image)
        label.append(label_dict[cat])
        C += 1

    print(f"Category: {cat}, Images Loaded: {C}")

# Convert data and labels to NumPy arrays (optional)
data = np.array(data)
label = np.array(label)

Category: ECG Images of Patient that have History of MI (172x12=2064), Images Loaded: 172
Category: ECG Images of Patient that have abnormal heartbeat (233x12=2796), Images Loaded: 233
Category: ECG Images of Myocardial Infarction Patients (240x12=2880), Images Loaded: 239
Category: Normal Person ECG Images (284x12=3408), Images Loaded: 284


In [5]:
data=np.array(data)
data.shape

(928, 224, 224, 3)

In [6]:
label=np.array(label)
label.shape

(928,)

In [7]:
import keras
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, Activation, BatchNormalization, Flatten,Conv2D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from tensorflow.keras.applications import EfficientNetB0,EfficientNetB3

In [8]:
num_classes = 5
label = keras.utils.to_categorical(label, num_classes)

In [9]:
model=EfficientNetB0(include_top=True,weights=None,input_shape=(224,224,3),classes=5)
model.summary()

Model: "efficientnetb0"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)  │ (None, 224, 224, 3)    │              0 │ -                      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ rescaling (Rescaling)     │ (None, 224, 224, 3)    │              0 │ input_layer[0][0]      │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ normalization             │ (None, 224, 224, 3)    │              7 │ rescaling[0][0]        │
│ (Normalization)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv_pad             │ (None, 225, 225, 3)    │              0 │ normalization[0][0]    │
│ (ZeroPadding2D)           │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_conv (Conv2D)        │ (None, 112, 112, 32)   │            864 │ stem_conv_pad[0][0]    │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_bn                   │ (None, 112, 112, 32)   │            128 │ stem_conv[0][0]        │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ stem_activation           │ (None, 112, 112, 32)   │              0 │ stem_bn[0][0]          │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_dwconv            │ (None, 112, 112, 32)   │            288 │ stem_activation[0][0]  │
│ (DepthwiseConv2D)         │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_bn                │ (None, 112, 112, 32)   │            128 │ block1a_dwconv[0][0]   │
│ (BatchNormalization)      │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_activation        │ (None, 112, 112, 32)   │              0 │ block1a_bn[0][0]       │
│ (Activation)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_squeeze        │ (None, 32)             │              0 │ block1a_activation[0]… │
│ (GlobalAveragePooling2D)  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reshape        │ (None, 1, 1, 32)       │              0 │ block1a_se_squeeze[0]… │
│ (Reshape)                 │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_reduce         │ (None, 1, 1, 8)        │            264 │ block1a_se_reshape[0]… │
│ (Conv2D)                  │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ block1a_se_expand         │ (None, 1, 1, 32)       │            288 │ block1a_se_reduce[0][… │
│ (Conv2D)                  │                        │                │                        │
├──────────────────────

 Total params: 4,055,976 (15.47 MB)

 Trainable params: 4,013,953 (15.31 MB)

 Non-trainable params: 42,023 (164.16 KB)

In [10]:
model.compile(loss='categorical_crossentropy', optimizer=Adam(0.0001),metrics=['accuracy'])

In [11]:
history=model.fit(data,label,batch_size=16,epochs=15, validation_split=0.1)

Epoch 1/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 144s 1s/step - accuracy: 0.3267 - loss: 2.2503 - val_accuracy: 1.0000 - val_loss: 1.3592
Epoch 2/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 34s 70ms/step - accuracy: 0.5309 - loss: 1.3582 - val_accuracy: 0.0000e+00 - val_loss: 1.3791
Epoch 3/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 5s 70ms/step - accuracy: 0.6818 - loss: 0.8196 - val_accuracy: 0.0000e+00 - val_loss: 1.5960
Epoch 4/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 72ms/step - accuracy: 0.7427 - loss: 0.6910 - val_accuracy: 0.0000e+00 - val_loss: 1.6845
Epoch 5/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 70ms/step - accuracy: 0.7412 - loss: 0.7465 - val_accuracy: 0.0000e+00 - val_loss: 2.7789
Epoch 6/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 71ms/step - accuracy: 0.8410 - loss: 0.4427 - val_accuracy: 1.0000 - val_loss: 0.6482
Epoch 7/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 74ms/step - accuracy: 0.8094 - loss: 0.5569 - val_accuracy: 0.0000e+00 - val_loss: 1.5158
Epoch 8/15
53/53 ━━━━━━━━━━━━━━━━━━━━ 4s 73ms/step - accuracy: 0.8296 - loss: 0.4301 - val

In [12]:
path = '/content/drive/MyDrive/kaggle/ECG_Images/test'

In [13]:
np.random.seed(1234)

img_list_test=os.listdir(path)
lables=[i for i in range(len(img_list_test))]
label_dict=dict()
label_dict['ECG Images of Myocardial Infarction Patients (240x12=2880)']= 0
label_dict['ECG Images of Patient that have History of MI (172x12=2064)']= 1
label_dict['ECG Images of Patient that have abnormal heartbeat (233x12=2796)']= 2
label_dict['Normal Person ECG Images (284x12=3408)']= 3
data_test=[]
label_test=[]
C=0
for cat in img_list_test:
    C=0
    pic_list=os.path.join(path,cat)
    for img in  os.listdir(pic_list):
        image=os.path.join(pic_list,img)
        #print(image)
        image=cv2.imread(image)
        #image=procssing(image)
        #image=cv2.cvtColor(image, cv2.COLOR_GRAY2BGR)
        image=cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, (224, 224))
        #image=img_to_array(image)
        data_test.append(image)
        label_test.append(label_dict[cat])
        C+=1
    print(C)

112
112
112
112


In [14]:
data_test=np.array(data_test)
data_test.shape

(448, 224, 224, 3)

In [15]:
label_test=np.array(label_test)
label_test.shape

(448,)

In [16]:
num_classes = 5
label_test = keras.utils.to_categorical(label_test, num_classes)
label_test.shape

(448, 5)

In [17]:
_, acc = model.evaluate(data_test, label_test)

14/14 ━━━━━━━━━━━━━━━━━━━━ 9s 38ms/step - accuracy: 0.9877 - loss: 0.0556


In [18]:
y_pred=model.predict(data_test)
y_pred=np.argmax(y_pred, axis=1)
y_pred

14/14 ━━━━━━━━━━━━━━━━━━━━ 6s 33ms/step


array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 2, 2, 2, 3, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       3, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 2, 2, 2, 2, 2, 2, 2, 3,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,